# Packages

In [1]:
pip install -r requirements_short_3.txt

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/7b/e5/83fcd7e9db036c179e0352bfcd20f81d728197a16f883e7b90307a88e65e/seaborn-0.13.0-py3-none-any.whl.metadata
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 27.8 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/e0/9e/80e2205d138960a49caea391f3710600895dd8292b6868dc9aff7aa593f9/scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/d0/0b/26ad95cf0b747be967b15fb71a06f5ac67aba0fd2f9cd174de6edefc4674/scikit_learn-1.3.2-cp310-cp3

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from multipolar import de_groot_models,polarization_visualization_tools
import numpy as np
from matplotlib import colors
import seaborn as sns

import pickle
import sklearn
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import networkx as nx
import pickle
# import graph_tool
# from graph_tool import topology as gt_topology
# from graph_tool import inference
import copy
import numpy as np
import imp
import pickle
import warnings

de_groot_models_paper = imp.load_source("de_groot_models","2019_10_26_de_groot_models.py")
misc_tools_paper = imp.load_source("misc_tools","2020_03_04_misc_tools.py")
polarization_visualization_tools_paper = imp.load_source("polarization_visualization_tools","2019_11_23_polarization_visualization_tools.py")

from sklearn.decomposition import PCA

/tmp/ipykernel_182/884034354.py:21: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


# Create appropriate files 

## Set parameters

In [3]:
run_name = 'inf_rate_test_4'

part_rate = 7

inf_rate = 400

act_rate = 5

ol_type = 'MPs_partyLeaders_OfficialAccounts_NonMPs'
# ol_type = 'MPs_partyLeaders_OfficialAccounts_NonMPs'

poles = 'legislative'


## Import dataset

In [10]:
# load retweet network (date,source,target,weight,day)
# Define column headers
column_headers = ['date', 'source', 'target', 'weight', 'day']

# Load retweet network with headers
df = pd.read_csv("./data/load/rt_net_unfiltered_2023-12-11_dup.csv", header=None, names=column_headers)


In [11]:
len(df)

16089093

## Activity dictionary (act_dct) - already created, doesn't need to change

In [3]:
rt_net = pd.read_csv('./data/load/rt_net_filtered_weighted_2023-12-11.csv',header=None)

In [ ]:
rt_net

In [4]:
# Convert to dictionary in the format '<source>': {'act': <weight>}
act_dict = {f"{row[0]}": {'act': row[2]} for _, row in rt_net.iterrows()}

In [5]:
with open('./data/load/act_dict_2023-12-10_dup.pkl', 'wb') as f:
    pickle.dump(act_dict, f)

In [6]:
len(act_dict)

824603

In [ ]:
rt_net

In [16]:
# # Search for rows where column 1 contains 'macron'
# search_result = rt_net[1].str.contains('EmmanuelMacron', case=False, na=False)

# # Filter the DataFrame based on the search result
# result_df = rt_net[search_result]

Who are the most active listeners/users?  

In [20]:
sorted_act_dict = dict(sorted(act_dict.items(), key=lambda item: item[1]['act'], reverse=True))

In [21]:
act_df = pd.DataFrame({'Key': list(sorted_act_dict.keys()), 'Value': [item['act'] for item in sorted_act_dict.values()]})

In [ ]:
act_df[0:20]

In [108]:
act_df.Value.describe()

count    824603.000000
mean          1.109683
std           1.327893
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max         449.000000
Name: Value, dtype: float64

In [20]:
act_df.to_excel('act_df_.xlsx')

## Choose your opinion leaders

In [6]:
# You first need to decide what you're filtering by for OLs - only MPs, MPs and official accounts etc - also what poles 

In [4]:
# load full list of OLs

# Read the 'all_mps' sheet from the Excel file
df_OLs = pd.read_excel('./data/twitter_handles/ols_gross_3.xlsx', sheet_name='OLs')

In [5]:
#including 'MPs_partyLeaders_OfficialAccounts_NonMPs'

df_OLs_baseline = df_OLs[(df_OLs['political group'].isin(['Ensemble', 'NUPES', 'LR', 'RN'])) & (df_OLs['handle'].notna())]

In [6]:
len(df_OLs_baseline)


421

In [7]:
df_OLs_baseline.groupby('political group').size()

political group
Ensemble    182
LR           83
NUPES       102
RN           54
dtype: int64

How many Opinion Leaders are present in our initial dataset?

In [ ]:
df

In [13]:
# Check if df_OLs_baseline['handle'] is in df['target']
handles_in_target = df_OLs_baseline['handle'].isin(df['target'])

# Count how many are and aren't in df['target']
count_in_target = handles_in_target.sum()
count_not_in_target = len(df_OLs_baseline) - count_in_target

# Print the results
print(f"Number of handles in df['target']: {count_in_target}")
print(f"Number of handles not in df['target']: {count_not_in_target}")

Number of handles in df['target']: 237
Number of handles not in df['target']: 184


In [ ]:
# Define the file path with OLs_retweeted_weight in the name
file_path = f"data/twitter_handles/OLs_by_retweets/OLs_initial_in_data_{run_name}.xlsx"

# Save the DataFrame to an Excel file
sum_per_target_sorted_with_group.to_excel(file_path, index=True, header=True, sheet_name='Sheet1')

#### Filter by influence threshold (in-directed retweets)

In [14]:
# how many in-directed retweets are needed to be an OL

####### filter by influence (1000 rt)
df_influential_users = df.copy()

df_influential_users = df_influential_users[df_influential_users.groupby('target')['weight'].transform('sum') > inf_rate]
df_influential_users = df_influential_users.reset_index(drop=True)


How many Opinion Leaders are influential based on influence threshold? 

In [15]:
# Extract the 'twitter_handle' values from df_elites and filtered_targets
twitter_handles_in_df = df_OLs_baseline['handle']
twitter_handles_in_target = df_influential_users['target']

# Find twitter handles that are in the target list
handles_found_in_target_INF_RATE = twitter_handles_in_df[twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

# Find twitter handles that are not in the target list
handles_not_found_in_target = twitter_handles_in_df[~twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

print("Opinion Leaders who are influential:", len(handles_found_in_target_INF_RATE))
print("Opinion Leaders who are not influential:", len(handles_not_found_in_target))

Opinion Leaders who are influential: 78
Opinion Leaders who are not influential: 343


How many OLs are left by party/political group?

In [16]:
df_OLs_baseline[df_OLs_baseline['handle'].isin(handles_found_in_target_INF_RATE)]['political group'].value_counts()

political group
Ensemble    32
NUPES       27
RN          10
LR           9
Name: count, dtype: int64

#### Filter by participation rate (days when an OL tweets)

In [17]:
part_rate

7

In [18]:
############################### A: Filter by number of days 
 # 70% participation rate   DAYS out of 25

# Group by 'day' and 'target' and count the occurrences
target_counts_per_day = df_influential_users.groupby(['day', 'target']).size().reset_index(name='count')
# Get the total number of unique days in the dataset
# total_days = df_influential_users['day'].nunique()


# Filter targets that appear at least once every day
filtered_targets = target_counts_per_day.groupby('target').filter(lambda x: len(x) >= part_rate)

############################## B: Filter by only 3 days before 1st and 2nd round elections 

# Extract the 'twitter_handle' values from df_elites and filtered_targets
twitter_handles_in_df = df_OLs_baseline['handle']
twitter_handles_in_target = filtered_targets['target']

# Find twitter handles that are in the target list
handles_found_in_target_PART_RATE = twitter_handles_in_df[twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

# Find twitter handles that are not in the target list
handles_not_found_in_target = twitter_handles_in_df[~twitter_handles_in_df.isin(twitter_handles_in_target)].tolist()

How many OLs are left by party/political group?

In [19]:
print("Handles found in the target list:", len(handles_found_in_target_PART_RATE))
print("Handles not found in the target list:", len(handles_not_found_in_target))
print(*handles_found_in_target_PART_RATE, sep=' ')

Handles found in the target list: 78
Handles not found in the target list: 343
AgnesRunacher al_petel ALeaument alexiscorbiere alma_dufour AQuatennens AurelienPradie Aurelientache avecvous BallardPhilippe benjaminhaddad brigitte_kuster BrunoLeMaire CarolineFiat54 CaronAymericoff CBeaune celinecalvez ChJacob77 ChristopheBechu Clem_Autain Clemence_Guette CZacharopoulou damienabad Deputee_Obono diaz_edwige E_DupondM ebothorel ECiotti EELV Elisabeth_Borne EmmanuelMacron enmarchefr ericcoquerel ericwoerth FJolivet36 franckallisio franckriester Francois_Ruffin FraPiquemal GabrielAttal GDarmanin GillesLeGendre GLarrive JLechanteux JLMelenchon jmzulesi JulienAubert84 JulienOdoul KarineLebon l_boyard ljacobelli mbompard mguevenoux MichelZumkeller MLP_officiel NMeizonnet OlgaGivernet oliviagregoire olivierdussopt olivierveran PaulVannierFI phdumont Portes_Thomas PrudhommeLoic RaquelGarridoFr SachaHoulie sebastienjumel sebchenu SebLecornu StanGuerini stephane1peu TrouveAurelie trudigoz vpecresse 

#### Final OLs after filtering

Combine the two - so we can see how many handles are left after filters

In [20]:
unique_handles = list(set(handles_found_in_target_PART_RATE) & set(handles_found_in_target_INF_RATE))

In [21]:
len(unique_handles)

78

In [22]:
df_OLs_baseline[df_OLs_baseline['handle'].isin(unique_handles)]['political group'].value_counts()

political group
Ensemble    32
NUPES       27
RN          10
LR           9
Name: count, dtype: int64

In [23]:
df_OLs_baseline[df_OLs_baseline['handle'].isin(unique_handles)]['political group']

4      Ensemble
5      Ensemble
7         NUPES
12        NUPES
16        NUPES
         ...   
461          LR
463       NUPES
465    Ensemble
470       NUPES
471          RN
Name: political group, Length: 78, dtype: object

Which OLs are the most retweeted? (is it MLP? Macron? Someone else?)

In [25]:
# Filter DataFrame based on 'target' column
OLs_retweeted = df[df['target'].isin(unique_handles)]

# Calculate the sum of 'weight' column for the filtered rows
OLs_retweeted_weight = OLs_retweeted.groupby('target')['weight'].sum()
OLs_retweeted_weight = OLs_retweeted_weight.sort_values(ascending=False)

print(OLs_retweeted_weight)

target
JLMelenchon        184440
MLP_officiel       163282
AQuatennens         50798
Portes_Thomas       45406
Francois_Ruffin     25801
                    ...  
jmzulesi              475
KarineLebon           459
E_DupondM             425
phdumont              422
GillesLeGendre        421
Name: weight, Length: 78, dtype: int64


In [26]:
OLs_retweeted_weight_by_pol_group = pd.merge(OLs_retweeted_weight, df_OLs_baseline[['handle', 'political group']], left_on='target', right_on='handle', how='left')

In [27]:
OLs_retweeted_weight_by_pol_group

,weight,handle,political group
0,184440,JLMelenchon,NUPES
1,163282,MLP_officiel,RN
2,50798,AQuatennens,NUPES
3,45406,Portes_Thomas,NUPES
4,25801,Francois_Ruffin,NUPES
...,...,...,...
73,475,jmzulesi,Ensemble
74,459,KarineLebon,NUPES
75,425,E_DupondM,Ensemble
76,422,phdumont,LR


In [28]:
# Define the file path with OLs_retweeted_weight in the name
file_path = f"data/twitter_handles/OLs_by_retweets/OLs_retweeted_weight_by_pol_group_{run_name}.xlsx"

# Save the DataFrame to an Excel file
OLs_retweeted_weight_by_pol_group.to_excel(file_path, index=True, header=True, sheet_name='Sheet1')

print(f"DataFrame saved to Excel file: {file_path}")


DataFrame saved to Excel file: data/twitter_handles/OLs_by_retweets/OLs_retweeted_weight_by_pol_group_inf_rate_test_4.xlsx


#### Save list of OLs as csv (handle) and excel (handle, affiliation,position)

In [30]:
import os
excel_file_path = f"./data/twitter_handles/{run_name}_OLs_retweeted.csv"
# Assuming OLs_retweeted_weight_by_pol_group['handle'] is a Series or DataFrame
OLs_retweeted_weight_by_pol_group['handle'].to_csv(excel_file_path, index=False, header=False)

In [31]:
excel_file_path = f"./data/twitter_handles/{run_name}_OLs_retweeted.xlsx"
OLs_retweeted_weight_by_pol_group.to_excel(excel_file_path, index=False)

## Create the poles dictionary

This is a dictionary with each of the 4 poles, containing handles of each OL

In [32]:
# open your previously saved OL list excel 
excel_file_path = f"./data/twitter_handles/{run_name}_OLs_retweeted.xlsx"
df_OL = pd.read_excel(excel_file_path)

# Replace values in the 'political group' column
replace_dict = {'NUPES': 'Nouvelle Union', 'RN': 'Rassemblement National', 'LR': 'Republicains'}
df_OL['political group'] = df_OL['political group'].replace(replace_dict)

# Create an empty dictionary to store Political Groups and Twitter handles
political_groups_dict = {}

# Iterate through the DataFrame and populate the dictionary
for index, row in df_OL.iterrows():
    political_group = row['political group']
    twitter_handle = row['handle']
    
    # Check if the political_group is already a key in the dictionary
    if political_group in political_groups_dict:
        # If yes, append the twitter_handle to the existing list
        political_groups_dict[political_group].append(twitter_handle)
    else:
        # If not, create a new key with a list containing the twitter_handle
        political_groups_dict[political_group] = [twitter_handle]

pol_gr_path = f"./data/load/{run_name}_poles_2023-12-11.pkl"

with open(pol_gr_path, 'wb') as f:
    pickle.dump(political_groups_dict, f)

# loading files

In [34]:
data_path_load="./data/load/"
nw_load="rt_net_filtered_2023-12-11_dup.csv"
poles_nodes_load=f"{run_name}_poles_2023-12-11.pkl"
act_dct_load="act_dict_2023-12-10_dup.pkl"

data_path_save="./data/save/"
elite_con_save=f"{run_name}_rt_network_econ_cd.csv"
elite_dct_save=f"{run_name}_elite_dct_cd.p"
degroot_save=f"{run_name}_degroot_cd.p"

fig_path_save="./figures/"
fig_2d_save=f"{run_name}_degroot_2d_projs"
fig_pca_save=f"{run_name}_pca_2d_projs"
fig_AvB_save=f"{run_name}_degroot_AvB"
fig_data_dir="fig_data/"
degroot_tol = 1e-1 ## For actual computation, we used 1e-6, but 1e-1 is enough
act_min = 5 # filter by amount of activit 

poles_corresp = {'Nouvelle Union':0,
    'Rassemblement National':1,
    "Ensemble":2,
    "Republicains":3}

In [35]:
## Load elite
with open(data_path_load+poles_nodes_load,"rb") as f:
    poles_nodes_dct = pickle.load(f)
## Check that there is no user overlap between parties 
all_usrs = set()
sum_len = 0
for party, usrs in poles_nodes_dct.items():
    all_usrs.update(usrs)
    sum_len += len(usrs)
assert sum_len == len(all_usrs)

In [36]:
## Load network
G = nx.read_weighted_edgelist(data_path_load+nw_load,delimiter=",",create_using=nx.DiGraph())

DESCRIBING THE NETWORK

In [ ]:
num_nodes = G.number_of_nodes()
print(num_nodes)

In [ ]:
num_edges = G.number_of_edges()
print(num_edges)

In [37]:
## Build dictionary of coordinates for elite nodes and list of elite nodes
elite_dct, elite_nodes, poles_coord = de_groot_models.assign_poles_to_nodes(poles_corresp,poles_nodes_dct)

## Show dictionary with coordinates of elite nodes
## These coordinates are obtained using de_groot_models.get_simplex_vertex(D) with D
## the number of dimensions, which for a system with n poles is n-1 (for n=3, we have D=2 dimensions).
elite_dct,elite_nodes,poles_coord

({'JLMelenchon': array([1., 0., 0.]),
  'AQuatennens': array([1., 0., 0.]),
  'Portes_Thomas': array([1., 0., 0.]),
  'Francois_Ruffin': array([1., 0., 0.]),
  'alexiscorbiere': array([1., 0., 0.]),
  'mbompard': array([1., 0., 0.]),
  'Clem_Autain': array([1., 0., 0.]),
  'CaronAymericoff': array([1., 0., 0.]),
  'ALeaument': array([1., 0., 0.]),
  'yjadot': array([1., 0., 0.]),
  'Clemence_Guette': array([1., 0., 0.]),
  'TrouveAurelie': array([1., 0., 0.]),
  'ericcoquerel': array([1., 0., 0.]),
  'RaquelGarridoFr': array([1., 0., 0.]),
  'Deputee_Obono': array([1., 0., 0.]),
  'l_boyard': array([1., 0., 0.]),
  'alma_dufour': array([1., 0., 0.]),
  'sebastienjumel': array([1., 0., 0.]),
  'EELV': array([1., 0., 0.]),
  'WilliamMartinet': array([1., 0., 0.]),
  'PrudhommeLoic': array([1., 0., 0.]),
  'CarolineFiat54': array([1., 0., 0.]),
  'stephane1peu': array([1., 0., 0.]),
  'Aurelientache': array([1., 0., 0.]),
  'PaulVannierFI': array([1., 0., 0.]),
  'FraPiquemal': array([1.,

In [38]:
#  this is to find listeners, those with at least a directed connection 
G_econ = de_groot_models.get_elite_connected(G, elite_nodes, max_path_len=10)
## Show size of elite-connected network and of the original network (for an undirected network they must be the same)
print ("Elite-con network:", G_econ.order(), G.order(), nx.is_directed(G_econ))

calculando las conexiones...
0 159092
1 409055
2 544210
3 626882
4 649197
5 652879
6 653445
7 653605
8 653607
9 653607
Elite-con network: 653612 870462 True


In [39]:
## Verify that the network is connected (althought not strongly connected)
G_nx_UD = misc_tools_paper.better_to_undirected(G_econ)
G_nx_UD_GCC = polarization_visualization_tools_paper.get_GCC(G_nx_UD)
assert G_nx_UD_GCC.order() == G_econ.order() 

In [40]:
## Save Elite-connected network
nx.write_weighted_edgelist(G_econ,data_path_save+elite_con_save,delimiter=",")
with open(data_path_save+elite_dct_save,"wb") as f:
    pickle.dump(elite_dct, f)

In [41]:
## Compute degroot
nodes_order= list(G_econ.nodes)
order_dict= dict([ (v,i) for i,v in enumerate(nodes_order) ])
degroot = de_groot_models.solve_model_Ax_iterative(
                G_econ,
                elite_dct,
                nodes_order = nodes_order,
                order_dict = order_dict,
                tol = degroot_tol
            )

## Save degroot
with open(data_path_save+degroot_save,"wb") as f:
    pickle.dump((nodes_order,order_dict,degroot),f)

Building adjacency matrix...
Initializing the model...
Solving polarization model with the iterative Ax method
0 inf
1 61869.15321943797
2 27936.088332244926
3 17444.831328262375
4 13321.259174562561
5 10015.96180928703
6 7929.595274358603
7 6523.966245270875
8 5479.923847550543
9 4872.660882073578
10 4446.413120116931
11 4183.667005719174
12 3993.973600015717
13 3859.8869563906796
14 3755.078458829242
15 3673.1571361072183
16 3604.6560592947985
17 3547.1479892458583
18 3496.2149663092714
19 3451.079119066876
20 3409.138737438094
21 3370.489438973309
22 3333.3950106741086
23 3298.207551834268
24 3263.748210352466
25 3230.389266607958
26 3197.4331533770446
27 3165.1523884722264
28 3133.09894691765
29 3101.4647976697474
30 3069.981289830139
31 3038.8136767419205
32 3007.7862140782004
33 2977.0346675060905
34 2946.4379222330404
35 2916.104603556517
36 2885.9505472300325
37 2856.0564025368767
38 2826.3614828953855
39 2796.926615041295
40 2767.7093753555573
41 2738.7573472030563
42 2710.038

In [42]:
# degroot
with open(data_path_save+degroot_save,"rb") as f:
    data_tuple = pickle.load(f)

# index the different parts of the tuple for easy access later 
nodes_order, order_dict, degroot= data_tuple

In [43]:
## Filter by activity

# act min = number of retweets or tweets made by the author  

with open(data_path_load+act_dct_load,"rb") as f:
    act_dct = pickle.load(f)
degroot_act_filter = []
kerr = 0
for i, n in enumerate(nodes_order):
    try:
        if act_dct[n]["act"] >= act_rate:
            degroot_act_filter.append(degroot[i])
    except KeyError:
        kerr += 1
print ("kerr",kerr)
print (len(degroot_act_filter), len(nodes_order))
degroot_act_filter = np.array(degroot_act_filter)
N_usrs = len(degroot_act_filter)
print ("Final number of users: ", N_usrs)

kerr 5
3818 653612
Final number of users:  3818


In [44]:
degroot_act_filter

array([[-0.09702808,  0.05646222,  0.22792395],
       [ 0.3173004 , -0.22369242,  0.25790957],
       [-0.33333333, -0.47140452,  0.81649658],
       ...,
       [ 0.00460495, -0.27083802,  0.30297469],
       [ 0.05532229, -0.31878227,  0.35575312],
       [ 0.05532229, -0.31878227,  0.35575312]])

In [45]:
type(degroot_act_filter)

numpy.ndarray

# Visualise all faces of tetrahedron

In [46]:
#  for visualisation here, we should use degroot_act_filter object for everything 

In [47]:
## Figure 2D projections (paper)

fig_name_dct = {True:"Close",False:"All","orthogonal":"Ort","from_pole":"FrP"}
proj_type = "orthogonal"
project_close_only = True
## Generate sufix name
suf1 = fig_name_dct[proj_type]
suf2 = fig_name_dct[project_close_only]
suf = "_" + suf1 + "_" + suf2

fig = polarization_visualization_tools_paper.vis_2d_projections(
degroot_act_filter,
show_2d_contours=True,
show_center=True,
show_barycenter=True,
show_global_pca=True,
project_close_only=True,
save_fig_data=fig_path_save+fig_data_dir+fig_2d_save+suf,
proj_type = proj_type,
)

fig.savefig(fig_path_save+fig_2d_save+suf+".pdf")
fig.savefig(fig_path_save+fig_2d_save+suf+".png",dpi=600,transparent=True)

Computing global polarization compass...
Explained variance:  [0.19653358 0.20483411 0.59863231]
Projecting 2D points to the sides of triangle...
Plotting 1D histogram...
Plotting 2D histogram...
Plotting 2D contours...
Computing 2D kernel...
Masking points outside triangle...
19937
Plotting contours...
Plotting polarization compass...
Projecting 2D points to the sides of triangle...
Plotting 1D histogram...
Plotting 2D histogram...
Plotting 2D contours...
Computing 2D kernel...
Masking points outside triangle...
19937
Plotting contours...
Plotting polarization compass...
Projecting 2D points to the sides of triangle...
Plotting 1D histogram...
Plotting 2D histogram...
Plotting 2D contours...
Computing 2D kernel...
Masking points outside triangle...
19959
Plotting contours...
Plotting polarization compass...
Projecting 2D points to the sides of triangle...
Plotting 1D histogram...
Plotting 2D histogram...
Plotting 2D contours...
Computing 2D kernel...
Masking points outside triangle...

# PCA stats + visualisation

In [49]:
from multipolar import polarization_visualization_tools
import importlib
importlib.reload(polarization_visualization_tools)

<module 'multipolar.polarization_visualization_tools' from '/work/multipolar_4/multipolar/polarization_visualization_tools.py'>

In [50]:
degroot_act_filter.shape

(3818, 3)

In [51]:
# latest run, with more elites, more accurate elites, and more data

with warnings.catch_warnings(): # Suppress UserWarnings for deprecated distplot
    warnings.simplefilter("ignore", category=UserWarning)
    _=polarization_visualization_tools.vis_2d_pca(
        degroot_act_filter,
        pca_to_plot = [0,1], ## Which PCA should be plotted
        dir_signs = [1,1], ## To change direction of axis if needed
        hist2d_bins = 70,
        grid_div = 100,
        nlevels = 20,
        pole_lbls = list(poles_corresp.keys()),
        xlim = None,
        ylim = None
    )

plt.savefig(f'./figures/pca/{run_name}_pca_1_2_degroot_v2.png')

Computing 2D kernel...


In [52]:
# latest run, with more elites, more accurate elites, and more data

with warnings.catch_warnings(): # Suppress UserWarnings for deprecated distplot
    warnings.simplefilter("ignore", category=UserWarning)
    _=polarization_visualization_tools.vis_2d_pca(
        degroot_act_filter,
        pca_to_plot = [1,2], ## Which PCA should be plotted
        dir_signs = [1,1], ## To change direction of axis if needed
        hist2d_bins = 70,
        grid_div = 100,
        nlevels = 20,
        pole_lbls = list(poles_corresp.keys()),
        xlim = None,
        ylim = None
    )

plt.savefig(f'./figures/pca/{run_name}_pca_2_3_degroot_v2.png')

Computing 2D kernel...


# PCA and Total Variation stats

PCA

In [53]:
# # open previously saved degroot file (so we don't have to re-run the calculations) 

# with open('./data/save/degroot_cd.p',"rb") as f:
#     data_tuple = pickle.load(f)

# # index the different parts of the tuple for easy access later 
# nodes_order, order_dict, degroot = data_tuple

# State what the PC1, PC2, PC3 etc are
# Assuming data is your multidimensional array dataset

# Apply PCA
pca = PCA(n_components=3)
principal_components = pca.fit_transform(degroot_act_filter)

# Print the first three principal components

#  this value here is the direction of relationship of each feautre or dimension. In PC1, the thrid feature has the strongest effect, 0.668, and is positive 
#  when the value of the third feature increases, the value of PC1 tends to increase as well

print("Principal Component 1 (PC1):", pca.components_[0])
print("Principal Component 2 (PC2):", pca.components_[1])
print("Principal Component 3 (PC3):", pca.components_[2])

# Get explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Print explained variance for each principal component
for i, ratio in enumerate(explained_variance_ratio, start=1):
    print(f"Explained Variance PC{i}: {ratio:.4f}")


Principal Component 1 (PC1): [-0.25406446  0.91688386 -0.30785587]
Principal Component 2 (PC2): [-0.14432089 -0.35067564 -0.92530972]
Principal Component 3 (PC3): [ 0.9563591   0.19065829 -0.22141971]
Explained Variance PC1: 0.5986
Explained Variance PC2: 0.2048
Explained Variance PC3: 0.1965


--------------

TV - total variation

In [54]:
#  degroot

# Calculate the covariance matrix
cov_matrix = np.cov(degroot, rowvar=False)

# Calculate the total variation as the trace of the covariance matrix
total_variation = np.trace(cov_matrix)

print("Total Variation:", total_variation)

Total Variation: 0.09037772992151508


In [55]:
#  degroot_act_filter 

# Calculate the covariance matrix
cov_matrix = np.cov(degroot_act_filter, rowvar=False)

# Calculate the total variation as the trace of the covariance matrix
total_variation = np.trace(cov_matrix)

print("Total Variation:", total_variation)

Total Variation: 0.2049774930873794
